<h2 style="color:yellow;background:blue;padding:5px"> Importing the neccessary libraries and modules </h2> 

In [1]:
import numpy as np # linear algebra
import pandas as pd
import cv2, gc
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Flatten
from keras.applications.vgg19 import VGG19
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

<h2 style="color:yellow;background:blue;padding:5px"> Setting Defaults And Preprocessing</h2>

In [2]:
input_size = 128
epoch = 20
batch_size = 500
input_shape = (input_size, input_size, 3)
path = '/kaggle/input/planets-dataset/planet/planet/'

In [3]:
gc.collect()
train_classes = pd.read_csv(f'{path}train_classes.csv')
train_classes = shuffle(train_classes, random_state=0)
sample_submission = pd.read_csv(f'{path}sample_submission.csv')
trad_sample_df = sample_submission[sample_submission.image_name.str.contains('file_')].copy()
sample_submission = sample_submission[sample_submission.image_name.str.contains('test_')]

In [4]:
s = train_classes.tags.str.split(' ').explode()
lb = MultiLabelBinarizer()
encoded = lb.fit_transform(s.values[:, None])
one_hot_df = pd.DataFrame(encoded.tolist(), columns=np.ravel(lb.classes_), dtype='int') \
                .groupby(s.index) \
                .sum()
one_hot_df['image_name'] = train_classes["image_name"].apply(lambda fn: fn+".jpg")
cols = ['image_name'] + list(np.ravel(lb.classes_))
train_classes = one_hot_df[cols].copy()
del one_hot_df, s, encoded, lb
trad_sample_df['image_name'] = trad_sample_df["image_name"].apply(lambda fn: fn+".jpg")
sample_submission['image_name'] = sample_submission["image_name"].apply(lambda fn: fn+".jpg")

In [5]:
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                             zoom_range=0.5, rotation_range=90,
                             rescale=1./255.)

<h2 style="color:yellow;background:blue;padding:5px">Defining Functions</h2>

In [6]:
def VGG19_Amazon_Model(input_shape=input_shape):
    gc.collect()
    base_model = VGG19(include_top=False, weights='imagenet',
                       input_shape=input_shape)
    model = Sequential()
    model.add(BatchNormalization(input_shape=input_shape))
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(17, activation='sigmoid'))
   
    return model


def return_model_name(k):
    return '/kaggle/working/model_'+str(k)+'.h5'


def train_model(df, k=5):
    gc.collect()
    model = VGG19_Amazon_Model()
    kf = KFold(n_splits=k, random_state=1, shuffle=True)
    fold = 1

    for train_index, val_index in kf.split(df.image_name):
        
        training_data = df.iloc[train_index]
        validation_data = df.iloc[val_index]
        
        train_generator=datagen.flow_from_dataframe(
                                            dataframe=training_data, directory=f'{path}/train-jpg/',
                                            x_col="image_name", y_col=cols[1:], batch_size=batch_size,
                                            seed=42, shuffle=True, class_mode="raw",
                                            target_size=(input_size, input_size))
        
        val_generator=datagen.flow_from_dataframe(
                                            dataframe=validation_data, directory=f'{path}/train-jpg/',
                                            x_col="image_name", y_col=cols[1:], batch_size=batch_size,
                                            seed=42, shuffle=True, class_mode="raw",
                                            target_size=(input_size, input_size))
        
        STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
        STEP_SIZE_VAL = val_generator.n//val_generator.batch_size
        
        opt = Adam(learning_rate=0.0001)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

        callback = [EarlyStopping(monitor='val_accuracy', patience=4, verbose=1),
                    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,
                                   cooldown=2, verbose=1),
                    ModelCheckpoint(return_model_name(fold), monitor='val_accuracy', 
                                    verbose=1, save_best_only=True, mode='max')]
        history = model.fit_generator(train_generator, 
                                      validation_data=val_generator,
                                      callbacks=callback, verbose=1, epochs=epoch) 
        fold += 1
        
    return val_generator


def predict_model(test_gen, k=5, batch_size=batch_size):
    model = VGG19_Amazon_Model()
    model.load_weights(filepath='/kaggle/working/model_2.h5')
    full_test = []
    p_test = model.predict_generator(test_gen, verbose=1)
    full_test.append(p_test)
    #full_test.append(p_test)

    for nfold in range(1,k+1):
        model.load_weights(filepath=return_model_name(nfold))
        p_test = model.predict_generator(test_gen, verbose=1)
        full_test.append(p_test)
    result = np.array(full_test[0])

    #result = np.array(full_test)
    
    for i in range(1, k):
        result += np.array(full_test[i])
        result = result / k
    
    result_bool = (result > 0.18)
    
#     for i in result:
#         if i > 0.18:
#             result_bool.append(True)
#         else:
#             result_bool.append(False)
            
    
    return result_bool.astype(int)


def generate_original_format(df):
    preds = []
    for i in tqdm(range(df.shape[0]), miniters=1000):
        a = df.iloc[[i]]
        pred_tag=[]
        for k in cols[1:]:
            if(a[k][i] == 1):
                pred_tag.append(k)
        preds.append(' '.join(pred_tag))

    df['tags'] = preds
    df['image_name'] = df['image_name'].apply(lambda x: x.split('.')[0])
    return df[['image_name', 'tags']]

In [7]:
val_generator = train_model(train_classes)
gc.collect()

2022-09-19 09:12:07.885522: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 09:12:07.983088: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 09:12:07.983933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-19 09:12:07.985158: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

80150528/80134624 [==============================] - 0s 0us/step
Found 32383 validated image filenames.
Found 8096 validated image filenames.


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-09-19 09:13:31.940365: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-09-19 09:13:38.830130: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


65/65 [==============================] - 463s 7s/step - loss: 0.2056 - accuracy: 0.0528 - val_loss: 0.2652 - val_accuracy: 0.1897

Epoch 00001: val_accuracy improved from -inf to 0.18972, saving model to /kaggle/working/model_1.h5
Epoch 2/20
65/65 [==============================] - 269s 4s/step - loss: 0.1324 - accuracy: 0.1126 - val_loss: 0.2308 - val_accuracy: 0.0655

Epoch 00002: val_accuracy did not improve from 0.18972
Epoch 3/20
65/65 [==============================] - 271s 4s/step - loss: 0.1188 - accuracy: 0.1202 - val_loss: 0.1846 - val_accuracy: 0.0566

Epoch 00003: val_accuracy did not improve from 0.18972
Epoch 4/20
65/65 [==============================] - 270s 4s/step - loss: 0.1146 - accuracy: 0.1143 - val_loss: 0.1399 - val_accuracy: 0.0940

Epoch 00004: val_accuracy did not improve from 0.18972
Epoch 5/20
65/65 [==============================] - 272s 4s/step - loss: 0.1112 - accuracy: 0.1153 - val_loss: 0.1231 - val_accuracy: 0.0846

Epoch 00005: val_accuracy did not im

6581

<h2 style="color:yellow;background:blue;padding:5px">preliminary Predictions</h2>

In [8]:
pred_val = predict_model(val_generator, 5)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


17/17 [==============================] - 48s 3s/step


In [9]:
preds = np.argmax(pred_val, axis=1)
vals = np.argmax(val_generator.labels, axis=1)

print('F2 = {}'.format(fbeta_score(vals, preds, beta=2, average='micro')))

F2 = 0.31846819024088946


<h2 style="color:yellow;background:blue;padding:5px"> Testing the model and making predictions</h2>

In [10]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test1_generator=test_datagen.flow_from_dataframe(
                                            dataframe=sample_submission, directory=f'{path}/test-jpg/',
                                            x_col="image_name", y_col=None, batch_size=8,
                                            seed=42, shuffle=False, class_mode=None, 
                                            target_size=(input_size, input_size))

result1 = predict_model(test1_generator, 5)
result1 = pd.DataFrame(result1, columns=cols[1:])
result1["image_name"]=test1_generator.filenames
result1 = generate_original_format(result1.copy())

Found 40669 validated image filenames.


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


5084/5084 [==============================] - 112s 22ms/step


100%|██████████| 40669/40669 [00:34<00:00, 1185.88it/s]


In [11]:
test2_generator=test_datagen.flow_from_dataframe(
                                            dataframe=trad_sample_df, 
                                            directory='../input/planets-dataset/test-jpg-additional/test-jpg-additional/',
                                            x_col="image_name", y_col=None, batch_size=8,
                                            seed=42, shuffle=False, class_mode=None, 
                                            target_size=(input_size, input_size))

result2 = predict_model(test2_generator, 5)
result2 = pd.DataFrame(result2, columns=cols[1:])
result2["image_name"]=test2_generator.filenames
result2 = generate_original_format(result2.copy())

Found 20522 validated image filenames.
2566/2566 [==============================] - 58s 22ms/step


100%|██████████| 20522/20522 [00:17<00:00, 1188.14it/s]


In [12]:
results = result1.append(result2, ignore_index=True)
results.to_csv("my_submission.csv",index=False)